In [2]:
# 导入包
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
%matplotlib inline

In [5]:
# 导入数据
org_data = pd.read_csv("data.csv", encoding = 'gbk')
print(org_data.shape)

(4754, 90)


In [4]:
org_data.head()

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [18]:
# 正样本比例25%，占比较低
org_data.groupby('status')['status'].count()

status
0    3561
1    1193
Name: status, dtype: int64

In [6]:
# 去掉第一列
org_data = org_data.ix[:,1:]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [40]:
# org_data.dtypes
org_data_count = pd.DataFrame(org_data.nunique())
# org_data_count[org_data_count == 0 | org_data_count == len(org_data)]

In [52]:
org_data_count[org_data_count.ix[:, 0] == 1].index

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


Index(['bank_card_no', 'source'], dtype='object')

In [53]:
org_data_count[org_data_count.ix[:, 0] == len(org_data)].index

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


Index(['custid', 'trade_no'], dtype='object')

In [54]:
org_data = org_data.drop(['bank_card_no', 'source', 'custid', 'trade_no'], axis = 1)

In [55]:
print(org_data.shape)

(4754, 85)


##  1、数据类型分析

In [7]:
org_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 89 columns):
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility                             4752 non-null float64
student_feature

## 4、缺失值处理

In [8]:
# 缺失占比统计:70个变量都有缺失！
missing_rate_t = 1 - org_data.describe().T['count'] / len(org_data)
missing_rate = missing_rate_t[missing_rate_t > 0].sort_values(ascending = False)
print(len(missing_rate))

70


In [10]:
missing_rate_50 = missing_rate_t[missing_rate_t > 0.5].sort_values(ascending = False)
print(missing_rate_50)

student_feature    0.630627
Name: count, dtype: float64


In [12]:
org_data1 = org_data.drop(missing_rate_50.index, axis=1)

In [16]:
#打印缺失占比
missing_rate_t1 = 1 - org_data1.describe().T['count'] / len(org_data)
missing_rate1 = missing_rate_t1[missing_rate_t1 > 0].sort_values(ascending = False)
print(missing_rate1)

cross_consume_count_last_1_month           0.089609
query_cash_count                           0.063946
latest_query_day                           0.063946
latest_six_month_apply                     0.063946
latest_one_month_apply                     0.063946
query_sum_count                            0.063946
latest_three_month_apply                   0.063946
query_finance_count                        0.063946
query_org_count                            0.063946
apply_credibility                          0.063946
apply_score                                0.063946
latest_one_month_loan                      0.062474
loans_score                                0.062474
loans_credibility_behavior                 0.062474
loans_count                                0.062474
loans_settle_count                         0.062474
loans_overdue_count                        0.062474
loans_org_count_behavior                   0.062474
consfin_org_count_behavior                 0.062474
loans_cash_c

In [ ]:
# 对数值缺失变量采用均值替换，对分类变量采用众数替换
var_num = org_data1.dtypes
var_char = 

if i in var_num:
    org_data1[i].fillna(org_data1[i].mean()[0],inplace = True)
    
if i in var_char:
    org_data1[i].fillna(data[i].mode()[0],inplace = True)


In [ ]:
# 再次打印缺失占比
missing_rate_t1 = 1 - org_data1.describe().T['count'] / len(org_data)
missing_rate1 = missing_rate_t1[missing_rate_t1 > 0].sort_values(ascending = False)
print(missing_rate1)

## 2、无关特征删除

In [22]:
# 连续变量——相关系数，将相关系数较低的变量删除


In [23]:
# 分类变量——卡方检验，检验不显著的删除

## 3、数据类型转换

In [ ]:
# 连续变量——查看分布



In [ ]:
# 分类变量——统计水平数
# 因不了解变量含义，若水平分类较多，按照最终分类频数均衡、对因变量的iv值最大对水平进行合并

In [ ]:
# 数据拆分
y = org_data1[var_y]
x = org_data1[var_num + var_char]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2018)